In [1]:
from herbert_emotion_classifier.wordnet_dataset_from_embeddings import EmotionsInTextSlowosieclineDataset
from herbert_emotion_classifier.baseline_dataset_from_embedding import EmotionsInTextBaselineDataset 

# from herbert_emotion_classifier.baseline_dataset_from_text import EmotionsInTextBaselineDataset 
# from herbert_emotion_classifier.wordnet_dataset_from_text import EmotionsInTextSlowosiecDataset 

from herbert_emotion_classifier.model import HerbertEmotionClassifier
from herbert_emotion_classifier.datamodule import EmotionsInTextDatamodule

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger

from os.path import join

import pandas as pd

In [2]:
model = HerbertEmotionClassifier()

In [3]:
train_ds = EmotionsInTextSlowosieclineDataset(join("..", "data", "slowosiec_all_data_embeddings.pickle"))
# train_ds = EmotionsInTextSlowosieclineDataset(join("..", "data", "wust_train_embeddings.pickle"))

val_ds = EmotionsInTextSlowosieclineDataset(join("..", "data", "wust_test_embeddings.pickle"))

In [4]:
# val_ds[0]

In [5]:
# train_ds = EmotionsInTextBaselineDataset()
# val_ds = EmotionsInTextBaselineDataset(join("..", "data", "wust_test.json"))

In [6]:
dm = EmotionsInTextDatamodule(16, train_ds, val_ds)

In [7]:
batch = next(iter(dm.val_dataloader()))

In [8]:
batch.get("sentence_embedding").size()

torch.Size([16, 768])

In [ ]:
# logger = WandbLogger(name="unet_resnet50",project="pst_segmentation")

seed_everything(42)

trainer = Trainer(max_epochs=100,
                  gpus=1,
                  deterministic=True,
                  accumulate_grad_batches=4,
                  checkpoint_callback=False,
                  callbacks=[EarlyStopping(monitor='val/loss')],
                  logger=False
                  )

trainer.fit(model, dm)

model.eval()
a=1
# model.freeze()


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | criterion  | CrossEntropyLoss | 0     
1 | classifier | Sequential       | 230 K 


In [ ]:
from herbert_emotion_classifier.predict import predict
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
import torch

def predict(model, dataloader, device):
    """
    :param model:
    :param dataloader:
    :param device:
    :param get_x_fun:
    :return:
    """
    model = model.to(device)

    num_elements = len(dataloader.dataset)
    num_batches = len(dataloader)

    batch_size = dataloader.batch_size

    predictions = torch.zeros(num_elements)
    true_labels = torch.zeros(num_elements)

    for i, batch in enumerate(dataloader):
        start = i * batch_size
        end = start + batch_size
        if i == num_batches - 1:
            end = num_elements

        x = batch["sentence_embedding"]
        y = batch["label"]
        logits = model.forward(x.to(device))
        pred = torch.argmax(logits, dim=1)
        predictions[start:end] = pred.detach()
        true_labels[start:end] = y.detach()

    return predictions, true_labels

In [ ]:
predictions, true_labels = predict(model, dm.val_dataloader(), "cuda")

In [ ]:
class_names = ['oczekiwanie', 'podziw', 'radosc', 'smutek', 'strach', 'wstret', 'zaskoczenie', 'zlosc', 'neutralne']

In [ ]:
cm = confusion_matrix(true_labels, predictions)
df_cm = pd.DataFrame(cm, index = [i for i in class_names],
                  columns = [i for i in class_names])

plt.figure(figsize = (10,7))

sn.heatmap(df_cm, cmap="Blues_r", annot=True, fmt="d")

In [ ]:
print(classification_report(true_labels, predictions, target_names=class_names))